In [1]:
#Import packages

#Rdkit
from rdkit import Chem

#Pandas
import pandas as pd
from pandas import DataFrame

#Others
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import joblib
import random
import numpy as np
import os
import math
import warnings

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *

In [2]:
def print_RxC(dataframe):
    rows= dataframe.shape[0]
    columns= dataframe.shape[1]
    rows_columns=print(dataframe.shape[0],"Moleculars ×",dataframe.shape[1],"Descriptors")
    return rows_columns
def trimm_correlated(df_in, threshold):
    df_corr = df_in.corr(method='spearman')
    df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > threshold).any()
    un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
    df_out = df_in[un_corr_idx]
    return df_out

In [3]:
#Setting the Work Path
warnings.filterwarnings ('ignore')
#Work Path
os.chdir("C:\\Users\\A\\Desktop\\Bioactive\\HSV-1")

In [4]:
#AlvaDesc
from alvadesccliwrapper.alvadesc import AlvaDesc
#AlvaDesc Path(Linux default alvaDescCLI location)
aDesc = AlvaDesc('E:\\C++\\alvaDesc\\alvaDescCLI.exe') 

In [5]:
Pubchem_data=pd.read_csv("./Pubchem_data.csv",sep=',')
Iso_smiles=Pubchem_data['Iso_smiles']
print_RxC(Pubchem_data)
Pubchem_data.head()

11406 Moleculars × 3 Descriptors


,cid,Iso_smiles,Type
0,5273,C1CC(OC1)[N+]2=CNC3=C(N=CN=C32)N,Adenosine
1,13463,C1=NC(=C2C(=N1)N(C=N2)C3C(C(C(O3)CCl)O)O)N,Adenosine
2,39148,CN(C)C1[C@H](O[C@H](C1O)N2C=NC3=C(N=CN=C32)N)CO,Adenosine
3,56328,C1[C@H](O[C@H](C1F)N2C=NC3=C(N=CN=C32)N)CO,Adenosine
4,67358,C1=NC(=C2C(=N1)N(C=[N+]2[O-])[C@H]3[C@@H]([C@@...,Adenosine


In [6]:
Smis=[]
for i in Iso_smiles:
    Smis.append(i)
Mols = []
for Smi in Smis:
    Mol = Chem.MolFromSmiles(Smi)
    Mols.append(Mol)
Canonical_smis = []
for Mol in Mols:
    Canonical_smi = Chem.MolToSmiles(Mol)
    Canonical_smis.append(Canonical_smi)
Pubchem_data['Canonical_smiles']=Canonical_smis

In [7]:
#Load the data
Raw_data= pd.read_csv('./Original_data.csv',index_col=0)
X_scaled_data= pd.read_csv('./X_scaled_data.csv',index_col=0)
Transformer= joblib.load(filename='./Scaler_transformer.pkl')

In [8]:
Pubchem_data=Pubchem_data.drop_duplicates(subset="cid")
Pubchem_data=Pubchem_data.drop_duplicates(subset="Canonical_smiles")
Pubchem_data=Pubchem_data[~ Pubchem_data['Canonical_smiles'].isin(Raw_data['Canonical_smiles'])]
Pubchem_data.index=Pubchem_data['cid']
Pubchem_data.drop('cid',axis = 1,inplace = True)
print_RxC(Pubchem_data)
Pubchem_data.to_csv("./Pubchem_data_clearn.csv",sep=',',header=1,index=1)
Pubchem_data.head()

7275 Moleculars × 3 Descriptors


,Iso_smiles,Type,Canonical_smiles
cid,,,
5273,C1CC(OC1)[N+]2=CNC3=C(N=CN=C32)N,Adenosine,Nc1ncnc2c1[nH]c[n+]2C1CCCO1
13463,C1=NC(=C2C(=N1)N(C=N2)C3C(C(C(O3)CCl)O)O)N,Adenosine,Nc1ncnc2c1ncn2C1OC(CCl)C(O)C1O
39148,CN(C)C1[C@H](O[C@H](C1O)N2C=NC3=C(N=CN=C32)N)CO,Adenosine,CN(C)C1C(O)[C@H](n2cnc3c(N)ncnc32)O[C@@H]1CO
56328,C1[C@H](O[C@H](C1F)N2C=NC3=C(N=CN=C32)N)CO,Adenosine,Nc1ncnc2c1ncn2[C@@H]1O[C@H](CO)CC1F
67358,C1=NC(=C2C(=N1)N(C=[N+]2[O-])[C@H]3[C@@H]([C@@...,Adenosine,Nc1ncnc2c1[n+]([O-])cn2[C@@H]1O[C@H](CO)[C@@H]...


In [9]:
X_scaled_data

,SAscore,WiA_D/Dt,J_D/Dt,SpMaxA_D/Dt,Chi_D/Dt,GATS3m,P_VSA_charge_1,NNRS,QED,RCI,...,MPC04,T(N..S),GATS7m,SpMax_D/Dt,EE_D/Dt,SM6_D/Dt,MATS1i,SM5_D/Dt,SpMAD_EA(ri),Eig06_AEA(bo)
cid,,,,,,,,,,,,,,,,,,,,,
596.0,0.483975,0.742006,0.142647,0.775040,0.074656,0.502577,0.299167,0.666667,0.161443,0.498667,...,0.635921,0.000000,0.392684,0.683875,0.681787,0.867268,0.516225,0.867604,0.395336,0.737895
1318.0,0.245427,0.359782,0.155196,0.367516,0.269726,0.462730,0.058868,0.500000,0.728096,0.528000,...,0.460867,0.030457,0.378666,0.126860,0.121143,0.365811,0.842371,0.367367,0.695037,0.590568
1570.0,0.397823,0.112213,0.281331,0.100821,0.603024,0.433900,0.377115,0.250000,0.385073,0.595294,...,0.632174,0.000000,0.519568,0.069399,0.064074,0.240680,0.738235,0.241921,0.662042,0.706296
1821.0,0.422965,0.355262,0.112934,0.349667,0.273123,0.453423,0.252221,0.666667,0.360650,0.502857,...,0.550857,0.000000,0.410171,0.143457,0.137826,0.395487,0.516805,0.397008,0.446648,0.623479
2199.0,0.275540,0.403391,0.134232,0.411549,0.224681,0.541115,0.108609,0.750000,0.589147,0.480000,...,0.624544,0.025381,0.423520,0.287360,0.282653,0.586467,0.644888,0.587516,0.618684,0.815787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135831485.0,0.226152,0.541122,0.114250,0.549391,0.142695,0.499139,0.147976,1.000000,0.701783,0.440000,...,0.583326,0.035533,0.553940,0.345468,0.341144,0.642827,0.530707,0.643732,0.371042,0.770861
135900555.0,0.238547,0.476069,0.095434,0.488765,0.186156,0.517294,0.051913,0.750000,0.357297,0.481905,...,0.592015,0.086294,0.474916,0.314540,0.310011,0.613864,0.580939,0.614838,0.420020,0.755659
135976680.0,0.204035,0.342174,0.093300,0.334971,0.285374,0.657759,0.039367,0.666667,0.834628,0.498667,...,0.485905,0.000000,0.400762,0.116555,0.110799,0.346155,0.614055,0.347742,0.684086,0.644623


In [10]:
Canonical_smis=Pubchem_data['Canonical_smiles']

In [11]:
#Calculate the descriotors
aDesc.set_input_SMILES(Canonical_smis.tolist())
if not aDesc.calculate_descriptors('ALL'):
 print('Error: ' + aDesc.get_error())
else:
 Data_all=aDesc.get_output()

In [12]:
Data_all=pd.DataFrame(Data_all)
Desc_names_names=aDesc.get_output_descriptors()
Data_all.columns=Desc_names_names
Data_all.index=Pubchem_data.index

In [13]:
X_test=Data_all[X_scaled_data.columns]
X_test.head()

,SAscore,WiA_D/Dt,J_D/Dt,SpMaxA_D/Dt,Chi_D/Dt,GATS3m,P_VSA_charge_1,NNRS,QED,RCI,...,MPC04,T(N..S),GATS7m,SpMax_D/Dt,EE_D/Dt,SM6_D/Dt,MATS1i,SM5_D/Dt,SpMAD_EA(ri),Eig06_AEA(bo)
cid,,,,,,,,,,,,,,,,,,,,,
5273,5.251602,0.409755,13.005587,0.390580,3.060138,0.940546,43.897075,0.666667,0.654958,1.142857,...,3.850148,0.0,1.156032,5.858706,5.892156,10.607949,-0.436922,8.838175,1.391659,2.298881
13463,4.559664,0.478692,13.526137,0.468925,2.576407,0.810527,129.263762,0.666667,0.567411,1.142857,...,4.110874,0.0,1.177161,8.909584,8.911602,13.122809,-0.381000,10.935367,1.378088,2.724409
39148,4.490448,0.511211,13.730854,0.503974,2.387975,0.960988,132.388076,0.666667,0.560992,1.142857,...,4.234107,0.0,1.004228,10.583453,10.583877,14.155770,-0.523113,11.796332,1.406512,2.925160
56328,4.490382,0.467042,13.314382,0.457973,2.662876,0.980411,86.580419,0.666667,0.757001,1.142857,...,4.043051,0.0,1.175066,8.243514,8.247174,12.656622,-0.329590,10.546770,1.387856,2.657115
67358,4.948142,0.478186,14.065831,0.468160,2.557424,0.889478,198.783243,0.666667,0.281631,1.142857,...,4.204693,0.0,1.157552,9.363199,9.364572,13.420757,-0.336745,11.183707,1.409046,2.724422


In [14]:
X_test_scaled=Transformer.transform(X_test)
X_test_scaled =pd.DataFrame(X_test_scaled)
X_test_scaled.columns=X_test.columns
X_test_scaled.index=X_test.index
X_test_scaled.to_csv("./X_test_data.csv",sep=',',header=1,index=1)
print_RxC(X_test_scaled)
X_test_scaled.head()

7275 Moleculars × 166 Descriptors


,SAscore,WiA_D/Dt,J_D/Dt,SpMaxA_D/Dt,Chi_D/Dt,GATS3m,P_VSA_charge_1,NNRS,QED,RCI,...,MPC04,T(N..S),GATS7m,SpMax_D/Dt,EE_D/Dt,SM6_D/Dt,MATS1i,SM5_D/Dt,SpMAD_EA(ri),Eig06_AEA(bo)
cid,,,,,,,,,,,,,,,,,,,,,
5273,0.568973,0.258075,0.095987,0.235567,0.373027,0.492642,0.078551,0.666667,0.668871,0.502857,...,0.454281,0.0,0.469021,0.057727,0.052288,0.209186,0.341907,0.211062,0.431500,0.524195
13463,0.416488,0.344728,0.107145,0.339526,0.285438,0.418994,0.231311,0.666667,0.569863,0.502857,...,0.535839,0.0,0.477594,0.128943,0.123236,0.369660,0.381909,0.371241,0.401896,0.621225
39148,0.401235,0.385603,0.111534,0.386034,0.251319,0.504221,0.236901,0.666667,0.562604,0.502857,...,0.574388,0.0,0.407432,0.168016,0.162530,0.435574,0.280251,0.436999,0.463901,0.667001
56328,0.401220,0.330083,0.102606,0.324993,0.301095,0.515223,0.154931,0.666667,0.784274,0.502857,...,0.514623,0.0,0.476744,0.113395,0.107624,0.339912,0.418685,0.341561,0.423204,0.605881
67358,0.502098,0.344091,0.118715,0.338511,0.282001,0.463715,0.355712,0.666667,0.246669,0.502857,...,0.565187,0.0,0.469638,0.139532,0.133880,0.388672,0.413566,0.390208,0.469428,0.621228
